# Velocity Data Download Script for Validation
#### Working Script to pull large velocity datasets and later compare to ICESAT2-derived velocities

ICESat-2 hackweek  
June 15, 2020  
Lynn Kaluzienski

# Import necessary modules

In [1]:
import os,re,h5py
import requests
import zipfile

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import scipy, sys, os, pyproj, glob
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [4]:
#! cd ..; [ -d pointCollection ] || git clone https://www.github.com/smithB/pointCollection.git
sys.path.append(os.path.join(os.getcwd(), '..'))

import pointCollection as pc
#stuck here, not sure how to import point collection 


In [5]:
#Magic function to enable interactive plotting (zoom/pan) in Jupyter notebook
#If running locally, this would be `%matplotlib notebook`, but since we're using Juptyerlab, we use widget
%matplotlib widget
#%matplotlib inline

In [6]:
'''
##Section where I played around with loading in .mat file, quickly abandoned
#load in data from .mat file
#Data originally in netcdf format, created subset in matlab for FIS region and saved as .mat file. 
import scipy.io as spio
data_root='/srv/surface_velocity/contributors/lynn_kaluzienski/Data_test'
#('Measures2.mat',squeeze_me=True)
Measures2_Files= spio.loadmat('./Data_test/Measures2.mat')
x=Measures2_Files['posx']
y=Measures2_Files['posy']
vx=Measures2_Files['vx_FIS']
vy=Measures2_Files['vy_FIS']
vel=Measures2_Files['vel_FIS']

#xy=np.array(pyproj.Proj(3031)(x, y))
plt.figure()
ax.imshow(vel, extent=[0, 1, 0, 1])
''';

In [8]:
#From Ben Smith's code loading in .tif file, running into issues likely with directories
data_root='/srv/shared/surface_velocity/FIS_Velocity/'
#spatial_extent = np.array([-102, -76, -98, -74.5])
spatial_extent = np.array([-65, -86, -55, -81])
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
#Originally tried to load data from a local directory, should change to shared directory
Measures_vel=pc.grid.data().from_geotif(os.path.join(data_root,'Measures2_FIS_Vel.tif'), bounds=[XR, YR])

# show the velocity map:
plt.figure()
Measures_vel.show(cmap='viridis', clim=[0,500])
plt.plot(xy[0,:], xy[1,:])
plt.title('Measures2 Velocity Map')


[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'viridis', 'clim': [0, 500], 'extent': array([-887878.65809562, -400567.81840096,  200333.90920048,
        561654.87344315]), 'origin': 'lower'}


Text(0.5, 1.0, 'Measures2 Velocity Map')

In [9]:
def atl06_to_dict(filename, beam, field_dict=None, index=None, epsg=None):
    """
        Read selected datasets from an ATL06 file

        Input arguments:
            filename: ATl06 file to read
            beam: a string specifying which beam is to be read (ex: gt1l, gt1r, gt2l, etc)
            field_dict: A dictinary describing the fields to be read
                    keys give the group names to be read, 
                    entries are lists of datasets within the groups
            index: which entries in each field to read
            epsg: an EPSG code specifying a projection (see www.epsg.org).  Good choices are:
                for Greenland, 3413 (polar stereographic projection, with Greenland along the Y axis)
                for Antarctica, 3031 (polar stereographic projection, centered on the Pouth Pole)
        Output argument:
            D6: dictionary containing ATL06 data.  Each dataset in 
                dataset_dict has its own entry in D6.  Each dataset 
                in D6 contains a numpy array containing the 
                data
    """
    if field_dict is None:
        field_dict={None:['latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}
    D={}
    file_re=re.compile('ATL06_(?P<date>\d+)_(?P<rgt>\d\d\d\d)(?P<cycle>\d\d)(?P<region>\d\d)_(?P<release>\d\d\d)_(?P<version>\d\d).h5')
    with h5py.File(filename,'r') as h5f:
        for key in field_dict:
            for ds in field_dict[key]:
                if key is not None:
                    ds_name=beam+'/land_ice_segments/'+key+'/'+ds
                else:
                    ds_name=beam+'/land_ice_segments/'+ds
                if index is not None:
                    D[ds]=np.array(h5f[ds_name][index])
                else:
                    D[ds]=np.array(h5f[ds_name])
                if '_FillValue' in h5f[ds_name].attrs:
                    bad_vals=D[ds]==h5f[ds_name].attrs['_FillValue']
                    D[ds]=D[ds].astype(float)
                    D[ds][bad_vals]=np.NaN
    if epsg is not None:
        xy=np.array(pyproj.proj.Proj(epsg)(D['longitude'], D['latitude']))
        D['x']=xy[0,:].reshape(D['latitude'].shape)
        D['y']=xy[1,:].reshape(D['latitude'].shape)
    temp=file_re.search(filename)
    D['rgt']=int(temp['rgt'])
    D['cycle']=int(temp['cycle'])
    D['beam']=beam
    return D

In [16]:
# Load a line and plot

data_root='/srv/shared/surface_velocity/'
field_dict={None:['delta_time','latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}

rgt = "0848"
cycle="03"
filename = glob.glob(os.path.join(data_root, 'FIS_ATL06', f'*ATL06_*_{rgt}{cycle}*_003*.h5'))

D=atl06_to_dict(filename[0],'/gt2l', field_dict=field_dict, index=None, epsg=3031)

# show the velocity map:
plt.figure()
Measures_vel.show(cmap='viridis', clim=[0,400])
plt.plot(xy[0,:], xy[1,:],'k')
plt.title('Measures2 Velocity Map')

plt.plot(D['x'],D['y'],'r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'viridis', 'clim': [0, 400], 'extent': array([-887878.65809562, -400567.81840096,  200333.90920048,
        561654.87344315]), 'origin': 'lower'}


In [44]:
# Interpolate the Measures velocities along the line and plot (note that these are speeds for now)

V = Measures_vel.interp(D['x'],D['y'], gridded=False)

plt.figure()
plt.plot(D['x_atc'],V)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# extract data velocity points

idea from
https://www.earthdatascience.org/courses/use-data-open-source-python/spatial-data-applications/lidar-remote-sensing-uncertainty/extract-data-from-raster/

In [46]:
import pandas as pd
import rasterio as rio
import geopandas as gpd


from geopandas import GeoDataFrame
from shapely.geometry import Point

In [24]:
#Dictionary to Pandas df
df = pd.DataFrame.from_dict(D)


In [25]:
#Convert points to shape file or geopandas

geometry = [Point(xy) for xy in zip(df.x, df.y)]
df = df.drop(['x', 'y'], axis=1)
crs = {'init': 'epsg:3031'}
gdf = GeoDataFrame(df, crs=crs, geometry=geometry)

/srv/conda/envs/notebook/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [31]:
#create shapefile

gdf.to_file('test_line_is2.shp', driver='ESRI Shapefile')

In [33]:
# bufffer over points

gdf.buffer(20).to_file('test_line_is2.shp')

is2_poly =gdf.copy()
is2_poly["geometry"] = gdf.geometry.buffer(40) # 40 meters buffer for zonal statitics
is2_poly.head()

,delta_time,latitude,longitude,h_li,atl06_quality_summary,x_atc,y_atc,dh_fit_dx,dh_fit_dy,rgt,cycle,beam,geometry
0,4.387628e+07,-81.000132,-55.500558,99.545357,0,2.912429e+07,36.549355,0.000697,-0.000276,848,3,/gt2l,"POLYGON ((-807437.572 554952.385, -807437.764 ..."
1,4.387628e+07,-81.000305,-55.500826,99.541039,0,2.912431e+07,36.477020,-0.004370,-0.000480,848,3,/gt2l,"POLYGON ((-807424.574 554937.880, -807424.766 ..."
2,4.387628e+07,-81.000478,-55.501095,99.530708,0,2.912433e+07,36.402977,0.002140,-0.000554,848,3,/gt2l,"POLYGON ((-807411.571 554923.378, -807411.764 ..."
3,4.387628e+07,-81.000652,-55.501362,99.531052,0,2.912435e+07,36.344711,0.000911,-0.000570,848,3,/gt2l,"POLYGON ((-807398.559 554908.886, -807398.752 ..."
4,4.387628e+07,-81.000825,-55.501629,99.546165,0,2.912437e+07,36.305172,0.004251,-0.000422,848,3,/gt2l,"POLYGON ((-807385.533 554894.405, -807385.726 ..."


In [35]:
# If the dir does not exist, create it
output_path = os.path.join("poly")

if not os.path.isdir(output_path):
    os.mkdir(output_path)

# Export the buffered point layer as a shapefile to use in zonal stats
plot_buffer_path = os.path.join(output_path, "plot_buffer.shp")
is2_poly.to_file(plot_buffer_path)


In [36]:
# Rasterstats contains the zonalstatistics function 
# that you will use to extract raster values
import rasterstats as rs

In [ ]:
# Extract zonal stats
Measures_vel_2=rio.open(os.path.join(data_root,'Measures2_FIS_Vel.tif'))

MeASUREs_vel2 = rs.zonal_stats(plot_buffer_path,
                                   SJER_chm_data,
                                   nodata=-999,
                                   affine=sjer_chm_meta['transform'],
                                   geojson_out=True,
                                   copy_properties=True,
                                   stats="count min mean max median")

# View object type
type(MeASUREs_vel)
